# ETL da camada Raw para camada Silver - Microsoft Security Incident Prediction

Este notebook realiza o técnico ETL (Extract, Transform, Load) dos dados da camada Raw para a camada Silver. 
Ele processa o dataset Microsoft Security Incident Prediction, realizando transformações, limpeza de dados e salvando os dados processados.


## EXTRACT

Nesta seção, extraímos os dados do arquivo CSV da camada Raw.


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Configuração do caminho dos dados
data_layer_filepath = '../../'

# Carregamento do dataset
df = pd.read_csv(data_layer_filepath + 'raw/train.csv', low_memory=False)
print("Dataset carregado com sucesso!")
print(f"Dimensões do dataset: {df.shape}")
df.head()


Dataset carregado com sucesso!
Dimensões do dataset: (9516837, 45)


,Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,...,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City
0,180388628218,0,612,123247,2024-06-04T06:05:15.000Z,7,6,InitialAccess,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,31,6,3
1,455266534868,88,326,210035,2024-06-14T03:01:25.000Z,58,43,Exfiltration,NaN,FalsePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
2,1056561957389,809,58352,712507,2024-06-13T04:52:55.000Z,423,298,InitialAccess,T1189,FalsePositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
3,1279900258736,92,32992,774301,2024-06-10T16:39:36.000Z,2,2,CommandAndControl,NaN,BenignPositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
4,214748368522,148,4359,188041,2024-06-15T01:08:07.000Z,9,74,Execution,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630


## TRANSFORM

Nesta seção, realizamos as transformações necessárias nos dados.


### Padronização dos Nomes das Colunas

Padronizamos os nomes das colunas para facilitar o processamento. O padrão será: **todos os caracteres em minúsculo, separando palavras com `_`**.


In [3]:
# Backup dos nomes originais
original_columns = df.columns.tolist()

# Padronização dos nomes das colunas
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

print("Primeiras 20 colunas padronizadas:")
for i, col in enumerate(df.columns[:20]):
    print(f"{i+1:2d}. {col}")

if len(df.columns) > 20:
    print(f"\n... e mais {len(df.columns) - 20} colunas")


Primeiras 20 colunas padronizadas:
 1. id
 2. orgid
 3. incidentid
 4. alertid
 5. timestamp
 6. detectorid
 7. alerttitle
 8. category
 9. mitretechniques
10. incidentgrade
11. actiongrouped
12. actiongranular
13. entitytype
14. evidencerole
15. deviceid
16. sha256
17. ipaddress
18. url
19. accountsid
20. accountupn

... e mais 25 colunas


### Análise de Valores Ausentes

Vamos analisar a presença de valores ausentes no dataset.


In [4]:
# Análise de valores nulos
null_counts = df.isnull().sum()
null_percentages = (null_counts / len(df)) * 100

# Criar DataFrame com informações de nulos
null_info = pd.DataFrame({
    'Coluna': null_counts.index,
    'Valores_Nulos': null_counts.values,
    'Percentual_Nulos': null_percentages.values
})

# Ordenar por percentual de nulos
null_info = null_info.sort_values('Percentual_Nulos', ascending=False)

print("Top 20 colunas com mais valores nulos:")
print(null_info.head(20).to_string(index=False))

# Identificar colunas com muitos valores ausentes (>50%)
high_missing_cols = null_info[null_info['Percentual_Nulos'] > 50]['Coluna'].tolist()
print(f"\nColunas com mais de 50% de valores ausentes: {len(high_missing_cols)}")
if high_missing_cols:
    print("Primeiras 10:", high_missing_cols[:10])


Top 20 colunas com mais valores nulos:
           Coluna  Valores_Nulos  Percentual_Nulos
     resourcetype        9509762         99.925658
    actiongrouped        9460773         99.410897
   actiongranular        9460773         99.410897
     threatfamily        9441956         99.213173
   emailclusterid        9420025         98.982729
antispamdirection        9339535         98.136965
            roles        9298686         97.707736
   suspicionlevel        8072708         84.825536
      lastverdict        7282572         76.523030
  mitretechniques        5468386         57.460120
    incidentgrade          51340          0.539465
        timestamp              0          0.000000
               id              0          0.000000
       entitytype              0          0.000000
       detectorid              0          0.000000
       alerttitle              0          0.000000
         category              0          0.000000
            orgid              0          0

### Remoção de Colunas com Muitos Valores Ausentes

Removemos colunas que têm mais de 80% de valores ausentes, pois não agregam valor significativo ao modelo.


In [5]:
# Definir threshold para remoção (80% de valores ausentes)
missing_threshold = 80

# Identificar colunas para remoção
cols_to_drop = null_info[null_info['Percentual_Nulos'] > missing_threshold]['Coluna'].tolist()

print(f"Colunas a serem removidas (>{missing_threshold}% ausentes): {len(cols_to_drop)}")
if cols_to_drop:
    print("Primeiras 10 colunas a serem removidas:")
    for i, col in enumerate(cols_to_drop[:10]):
        print(f"  {i+1}. {col}")

# Remover colunas com muitos valores ausentes
df_cleaned = df.drop(columns=cols_to_drop)
print(f"\nDimensões após remoção: {df_cleaned.shape}")
print(f"Colunas removidas: {df.shape[1] - df_cleaned.shape[1]}")

# Atualizar DataFrame principal
df = df_cleaned


Colunas a serem removidas (>80% ausentes): 8
Primeiras 10 colunas a serem removidas:
  1. resourcetype
  2. actiongrouped
  3. actiongranular
  4. threatfamily
  5. emailclusterid
  6. antispamdirection
  7. roles
  8. suspicionlevel

Dimensões após remoção: (9516837, 37)
Colunas removidas: 8


### Análise da Variável Target

Vamos analisar a variável target `HasDetections` para entender o balanceamento das classes.


In [6]:
# Análise da variável target
if 'hasdetections' in df.columns:
    target_counts = df['hasdetections'].value_counts()
    target_percentages = df['hasdetections'].value_counts(normalize=True) * 100
    
    print("Distribuição da variável target (HasDetections):")
    print(f"\nContagem:")
    for value, count in target_counts.items():
        print(f"  {value}: {count:,} ({target_percentages[value]:.2f}%)")
    
    # Verificar balanceamento
    balance_ratio = min(target_counts) / max(target_counts)
    print(f"\nRazão de balanceamento: {balance_ratio:.3f}")
    if balance_ratio < 0.1:
        print("⚠️  Dataset altamente desbalanceado!")
    elif balance_ratio < 0.3:
        print("⚠️  Dataset moderadamente desbalanceado")
    else:
        print("✅ Dataset relativamente balanceado")
        
    # Verificar valores ausentes na target
    target_missing = df['hasdetections'].isnull().sum()
    print(f"\nValores ausentes na variável target: {target_missing}")
    
else:
    print("Variável 'hasdetections' não encontrada no dataset.")
    print("Verificando colunas que podem ser a target:")
    potential_targets = [col for col in df.columns if 'detect' in col.lower() or 'target' in col.lower() or 'label' in col.lower()]
    if potential_targets:
        print(potential_targets)
    else:
        print("Nenhuma coluna target óbvia encontrada.")


Variável 'hasdetections' não encontrada no dataset.
Verificando colunas que podem ser a target:
['detectorid']


### Encoding de Variáveis Categóricas

Vamos aplicar Label Encoding para variáveis categóricas.


In [7]:
from sklearn.preprocessing import LabelEncoder

# Identificar colunas categóricas
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Remover a variável target se ela for categórica
if 'hasdetections' in categorical_cols:
    categorical_cols.remove('hasdetections')

print(f"Colunas categóricas identificadas: {len(categorical_cols)}")

# Aplicar Label Encoding para colunas categóricas
label_encoders = {}
for col in categorical_cols:
    if col in df.columns:
        # Verificar número de categorias únicas
        unique_values = df[col].nunique()
        print(f"{col}: {unique_values} valores únicos")
        
        # Aplicar Label Encoding
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

print(f"\nEncoding aplicado em {len(label_encoders)} colunas categóricas.")


Colunas categóricas identificadas: 7
timestamp: 760944 valores únicos
category: 20 valores únicos
mitretechniques: 1193 valores únicos
incidentgrade: 3 valores únicos
entitytype: 33 valores únicos
evidencerole: 2 valores únicos
lastverdict: 5 valores únicos

Encoding aplicado em 7 colunas categóricas.


### Tratamento de Valores Ausentes

Vamos tratar os valores ausentes restantes usando estratégias apropriadas.


In [8]:
# Verificar valores ausentes após encoding
missing_after_encoding = df.isnull().sum()
cols_with_missing = missing_after_encoding[missing_after_encoding > 0]

print("Valores ausentes após encoding:")
if len(cols_with_missing) > 0:
    print(cols_with_missing.sort_values(ascending=False))
    
    # Estratégias de tratamento
    for col in cols_with_missing.index:
        if df[col].dtype in ['int64', 'float64']:
            # Para colunas numéricas: imputação com mediana
            median_value = df[col].median()
            df[col].fillna(median_value, inplace=True)
            print(f"  {col}: Preenchido com mediana ({median_value})")
        else:
            # Para colunas categóricas: imputação com moda
            mode_value = df[col].mode()[0] if len(df[col].mode()) > 0 else 'Unknown'
            df[col].fillna(mode_value, inplace=True)
            print(f"  {col}: Preenchido com moda ({mode_value})")
else:
    print("Nenhum valor ausente encontrado!")

# Verificação final
final_missing = df.isnull().sum().sum()
print(f"\nTotal de valores ausentes após tratamento: {final_missing}")


Valores ausentes após encoding:
Nenhum valor ausente encontrado!

Total de valores ausentes após tratamento: 0


## LOAD

Nesta seção, carregamos os dados processados para a camada Silver.


### Salvando os Dados Processados em CSV

Vamos salvar os dados processados na camada Silver.


In [9]:
# Criar diretório silver se não existir
import os
silver_dir = data_layer_filepath + 'silver/'
if not os.path.exists(silver_dir):
    os.makedirs(silver_dir)
    print(f"Diretório criado: {silver_dir}")

# Salvar dados processados
output_file = silver_dir + 'security_incident_prediction_silver.csv'
df.to_csv(output_file, index=False)

print(f"Dataset da camada Silver salvo com sucesso!")
print(f"Arquivo: {output_file}")
print(f"Dimensões: {df.shape}")
print(f"Tamanho do arquivo: {os.path.getsize(output_file) / 1024**2:.2f} MB")


Dataset da camada Silver salvo com sucesso!
Arquivo: ../../silver/security_incident_prediction_silver.csv
Dimensões: (9516837, 37)
Tamanho do arquivo: 1715.69 MB


### Verificação Final da Qualidade dos Dados

Vamos fazer uma verificação final da qualidade dos dados processados.


In [10]:
# Verificação final da qualidade dos dados
print("=== VERIFICAÇÃO FINAL DA QUALIDADE DOS DADOS ===")
print(f"Dimensões finais do dataset: {df.shape}")
print(f"Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Verificar valores ausentes
missing_final = df.isnull().sum().sum()
print(f"Total de valores ausentes: {missing_final}")

# Verificar tipos de dados
print(f"\nTipos de dados finais:")
print(df.dtypes.value_counts())

# Verificar variável target
if 'hasdetections' in df.columns:
    target_dist = df['hasdetections'].value_counts(normalize=True) * 100
    print(f"\nDistribuição da variável target:")
    print(target_dist)

# Verificar se há valores infinitos
inf_count = np.isinf(df.select_dtypes(include=[np.number])).sum().sum()
print(f"\nValores infinitos encontrados: {inf_count}")

print("\n✅ Verificação de qualidade concluída!")


=== VERIFICAÇÃO FINAL DA QUALIDADE DOS DADOS ===
Dimensões finais do dataset: (9516837, 37)
Memória utilizada: 2686.49 MB
Total de valores ausentes: 0

Tipos de dados finais:
int64    37
Name: count, dtype: int64

Valores infinitos encontrados: 0

✅ Verificação de qualidade concluída!
